In [ ]:
# Going to make a data frame to wip which is linked on date to any dataframe to get adjusted values
# move it into toolbox where it will be in make data

In [ ]:
project_path = "/home/jupyter"
import sys

sys.path.append(project_path)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t

In [ ]:
client = bigquery.Client()

In [ ]:
# We first need to create our adjustment tables that we will bring in every time we want to create an adjusted value
# we do one for each table and each month/quarter then join on the date

# THE ADJUSTMENT TABLES will have to be amendended as they don't account for

In [ ]:
# sml

In [ ]:
%%bigquery
CREATE OR REPLACE VIEW ons-fintrans-analysis-prod.fin_wip_notebook.sml_cardholders_q AS
SELECT DATETIME(CAST(SUBSTRING(time_period_value, 1,4) AS int), CAST(SUBSTRING(time_period_value,6,6)AS int)*3, 01,00,00,00) AS
date_time, time_period_value, cardholders AS cardholders_all, FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location WHERE time_period = 'Quarter' AND 
mcg = 'All' AND merchant_location_level = 'All' AND cardholder_issuing_level = 'All'

In [ ]:
%%bigquery
CREATE OR REPLACE VIEW ons-fintrans-analysis-prod.fin_wip_notebook.sml_cardholders_m AS
SELECT DATETIME(CAST(SUBSTRING(time_period_value, 1,4) AS int), CAST(SUBSTRING(time_period_value,5,6)AS int), 01,00,00,00) AS
date_time, time_period_value, cardholders AS cardholders_all, FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location WHERE time_period = 'Month' AND 
mcg = 'All' AND merchant_location_level = 'All' AND cardholder_issuing_level = 'All'

In [ ]:
# spoc

In [ ]:
%%bigquery
CREATE OR REPLACE VIEW ons-fintrans-analysis-prod.fin_wip_notebook.spoc_cardholders_q AS
SELECT DATETIME(CAST(SUBSTRING(time_period_value, 1,4) AS int), CAST(SUBSTRING(time_period_value,6,6)AS int)*3, 01,00,00,00) AS
date_time, time_period_value, cardholders AS cardholders_all, FROM ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel WHERE time_period = 'Quarter' AND 
mcg = 'All' AND merchant_channel = 'All' AND cardholder_origin = 'All' AND destination_country = 'UNITED KINGDOM'

In [ ]:
%%bigquery
CREATE OR REPLACE VIEW ons-fintrans-analysis-prod.fin_wip_notebook.spoc_cardholders_m AS
SELECT DATETIME(CAST(SUBSTRING(time_period_value, 1,4) AS int), CAST(SUBSTRING(time_period_value,5,6)AS int), 01,00,00,00) AS
date_time, time_period_value, cardholders AS cardholders_all, FROM ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel WHERE time_period = 'Month' AND 
mcg = 'All' AND merchant_channel = 'All' AND cardholder_origin = 'All' AND destination_country = 'UNITED KINGDOM'

In [ ]:
# rphst

In [ ]:
%%bigquery
CREATE OR REPLACE VIEW ons-fintrans-analysis-prod.fin_wip_notebook.sml_cardholders_q AS
SELECT DATETIME(CAST(SUBSTRING(time_period_value, 1,4) AS int), CAST(SUBSTRING(time_period_value,6,6)AS int)*3, 01,00,00,00) AS
date_time, time_period_value, cardholders AS cardholders_all, FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location WHERE time_period = 'Quarter' AND 
mcg = 'All' AND merchant_location_level = 'All' AND cardholder_issuing_level = 'All'

In [ ]:
%%bigquery
CREATE OR REPLACE VIEW ons-fintrans-analysis-prod.fin_wip_notebook.sml_cardholders_m AS
SELECT DATETIME(CAST(SUBSTRING(time_period_value, 1,4) AS int), CAST(SUBSTRING(time_period_value,5,6)AS int), 01,00,00,00) AS
date_time, time_period_value, cardholders AS cardholders_all, FROM ons-fintrans-data-prod.fintrans_visa.spend_merchant_location WHERE time_period = 'Month' AND 
mcg = 'All' AND merchant_location_level = 'All' AND cardholder_issuing_level = 'All'

In [ ]:
def create_adjusted_value(df1, table, time_period):
    """ """
    df_adj = bq.read_full_bq_table(
        client,
        f"ons-fintrans-analysis-prod.fin_wip_notebook.{retrieve_adjustment_table(table,time_period)}",
    )
    # link on datetime first
    try:
        df = df1.merge(
            df_adj[["date_time", "cardholders_all"]], on="date_time", how="outer"
        )
    except Exception as e:
        print(
            f"{e}: using time_period_value instead of date_time, consider converting to date_time"
        )
        df = df1.merge(
            df_adj[["time_period_value", "cardholders_all"]],
            on="date_time",
            how="outer",
        )
    try:
        df["adj_spend"] = df["spend"] / df["cardholders_all"]
    except Exception as e:
        print(f"{e}")
    try:
        df["adj_transactions"] = df["transactions"] / df["cardholders_all"]
    except Exception as e:
        print(f"{e}")

    return df

In [ ]:
def retrieve_adjustment_table(table, time_period):
    """ """
    if time_period in ["Quarter", "quarter", "q"]:
        time_period = "Q"
    if time_period in ["Month", "month", "m"]:
        time_period = "M"

    if table == "spend_merchant_location":
        table = "sml"
    if table == "retail_performance_high_streets_towns":
        table = "rphst"
    if table == "spend_origin_and_channel":
        table = "spoc"

    return {
        ("sml", "Q"): "sml_cardholders_q",
        ("sml", "M"): "sml_cardholders_m",
        ("spoc", "Q"): "spoc_cardholders_q",
        ("spoc", "M"): "spoc_cardholders_m",
        ("rphst", "Q"): "rphst_cardholders_q",
        ("rphst", "M"): "rphst_cardholders_m",
    }[table, time_period]

In [ ]:
# showing how retrieve adjustment table works
retrieve_adjustment_table("sml", "M")

In [ ]:
# how retrieve adjustment function is used to retrieve and you can see the plot of cardholders
df_spoc = bq.read_full_bq_table(
    client,
    f"ons-fintrans-analysis-prod.fin_wip_notebook.{retrieve_adjustment_table('spoc','M')}",
)
plt.plot("date_time", "cardholders_all", data=df_spoc.sort_values("date_time"))
# The plot shows we may need to work on these tables and adjust differently i.e. use vicky's tables but
# we could end up having one adjustment table and adjusting like that
# Theoretically could probably join within the sql statement for efficiency purposes rather than via pandas

In [ ]:
df_check = bq.read_full_bq_table(
    client, "ons-fintrans-analysis-prod.fin_wip_notebook.sml_cardholders_q"
)

In [ ]:
df_check

In [ ]:
df1 = t.read_spend_merchant_location(client, mcg="")

In [ ]:
df1

In [ ]:
df3 = create_adjusted_value(df1, "sml", "Quarter")

In [ ]:
df3 = t.create_index(df3, "adj_spend", t.get_cat_vars("sml"))

In [ ]:
df3

In [ ]:
plt.plot(
    "date_time",
    "adj_spend_index",
    data=df3[df3["mcg"] == "TRAVEL SERVICES"].sort_values("date_time"),
)